<a href="https://colab.research.google.com/github/gokhangunal/RFM_Analysis_FLO_Customer_Segmentation/blob/main/RFM_Analysis_FLO_Customer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#An online shoe store called FLO wants to segment its customers and develop marketing 
#strategies based on these segments. For this purpose, customer behaviors will be identified 
#and groups will be formed based on the clustering of these behaviors.

In [ ]:
#master_id: Unique customer number
#order_channel: The channel used for the purchase (Android, ios, Desktop, Mobile)
#last_order_channel: The channel used for the last purchase
#first_order_date: The date of the customer's first purchase
#last_order_date: The date of the customer's last purchase
#last_order_date_online: The date of the customer's last online purchase
#last_order_date_offline: The date of the customer's last offline purchase
#order_num_total_ever_online: The total number of purchases the customer has made online
#order_num_total_ever_offline: The total number of purchases the customer has made offline
#customer_value_total_ever_offline: The total amount paid by the customer for offline purchases
#customer_value_total_ever_online: The total amount paid by the customer for online purchases
#interested_in_categories_12: The list of categories in which the customer made purchases in the last 12 months

In [ ]:
#1. Read the data and create a copy of the dataframe.
#2. Examine the first 10 observations, variable names, descriptive statistics, missing values, and variable types in the dataset.
#3. Create new variables for the total number of purchases and spending for each customer, 
#indicating omnichannel customers who have made purchases from both online and offline platforms.
#4. Examine variable types. Convert date variables to date format.
#5. Examine the distribution of the number of customers, total number of products purchased, and total spending across different shopping channels.
#6. Rank the top 10 customers with the highest revenue.
#7. Rank the top 10 customers with the highest number of orders.
#8. Functionize the data preprocessing steps.

In [ ]:
#1. Define Recency, Frequency, and Monetary metrics.
#2. Calculate Recency, Frequency, and Monetary metrics for each customer.
#3. Assign the calculated metrics to a variable named "rfm".
#4. Rename the metrics as recency, frequency, and monetary.

In [ ]:
# 1. Convert Recency, Frequency, and Monetary metrics into scores between 1-5 using qcut function.
# 2. Save these scores as recency_score, frequency_score, and monetary_score.
# 3. Combine recency_score and frequency_score into a single variable called RF_SCORE.

In [ ]:
# 1. Define segments for the created RF scores.
# 2. Convert the scores into segments using the seg_map.

In [ ]:
# 1. Examine the averages of recency, frequency, and monetary for each segment.
# 2. Using RFM analysis, find the customers in the relevant profile for the following 2 cases and save their customer IDs as a csv file:
      #   a. FLO is adding a new women's shoe brand to its portfolio. The prices of this brand's products are above the general customer preferences. 
      # Therefore, they want to communicate with the customers who are interested in this profile and make special efforts to promote the brand and sell its products. 
      # The loyal customers (champions, loyal_customers) and those who shop in the women's category will be the targeted customers for communication. 
      # Save the customer IDs of these customers to a csv file.
      #   b. A discount of nearly 40% is planned for men's and children's products. The company wants to target customers who are interested in these categories, 
      # but haven't made a purchase in a long time, as well as new customers. Save the customer IDs of these customers who are in the appropriate profile to a csv file.

In [1]:
!pip install lifetimes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.4 MB/s eta 0:00:00


In [2]:
import datetime as dt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.float_format',lambda x: '%.4f'%x)
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

In [ ]:
#1. Read the data and create a copy of the dataframe.

In [4]:
df_ = pd.read_csv("flo_data_20k.csv")

In [11]:
df = df_.copy()

In [7]:
#2. Examine the first 10 observations, variable names, descriptive statistics, missing values, and variable types in the dataset.

In [17]:
df.head(8)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0000,2.0000,159.9700,1853.5800,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR]
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,Desktop,Offline,2018-11-18,2021-03-13,2018-11-18,2021-03-13,1.0000,2.0000,150.8700,49.9900,[KADIN]
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,Android App,Android App,2020-03-04,2020-10-18,2020-10-18,2020-03-04,3.0000,1.0000,59.9900,315.9400,[AKTIFSPOR]
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,Mobile,Offline,2020-05-15,2020-08-12,2020-05-15,2020-08-12,1.0000,1.0000,49.9900,113.6400,[COCUK]


In [14]:
df.columns

Index(['master_id', 'order_channel', 'last_order_channel', 'first_order_date',
       'last_order_date', 'last_order_date_online', 'last_order_date_offline',
       'order_num_total_ever_online', 'order_num_total_ever_offline',
       'customer_value_total_ever_offline', 'customer_value_total_ever_online',
       'interested_in_categories_12'],
      dtype='object')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [27]:
df.shape

(19945, 12)

In [41]:
df.isnull().sum()

master_id                            0
order_channel                        0
last_order_channel                   0
first_order_date                     0
last_order_date                      0
last_order_date_online               0
last_order_date_offline              0
order_num_total_ever_online          0
order_num_total_ever_offline         0
customer_value_total_ever_offline    0
customer_value_total_ever_online     0
interested_in_categories_12          0
total_order                          0
total_customer_value                 0
dtype: int64

In [25]:
df["master_id"].nunique()

19945

In [16]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.0000,3.1109,4.2256,1.0000,1.0000,2.0000,4.0000,200.0000
order_num_total_ever_offline,19945.0000,1.9139,2.0629,1.0000,1.0000,1.0000,2.0000,109.0000
customer_value_total_ever_offline,19945.0000,253.9226,301.5329,10.0000,99.9900,179.9800,319.9700,18119.1400
customer_value_total_ever_online,19945.0000,497.3217,832.6019,12.9900,149.9800,286.4600,578.4400,45220.1300


In [19]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.25)
    quartile3 = dataframe[variable].quantile(0.75)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [21]:
outlier_thresholds(df,"order_num_total_ever_online")

(-3.5, 8.5)

In [22]:
outlier_thresholds(df,"order_num_total_ever_offline")

(-0.5, 3.5)

In [23]:
outlier_thresholds(df,"customer_value_total_ever_offline")

(-229.98000000000002, 649.94)

In [24]:
outlier_thresholds(df,"customer_value_total_ever_online")

(-492.71000000000004, 1221.13)

In [ ]:
#3. Create new variables for the total number of purchases and spending for each customer, 
#indicating omnichannel customers who have made purchases from both online and offline platforms.

In [28]:
df['total_order'] = df['order_num_total_ever_online'] + df['order_num_total_ever_offline']
df['total_customer_value'] = df['customer_value_total_ever_offline'] + df['customer_value_total_ever_online']

df.head(7)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order,total_customer_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN],5.0000,939.3700
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0000,2.0000,159.9700,1853.5800,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0000,2013.5500
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]",5.0000,585.3200
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]",2.0000,121.9700
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR],2.0000,209.9800
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,Desktop,Offline,2018-11-18,2021-03-13,2018-11-18,2021-03-13,1.0000,2.0000,150.8700,49.9900,[KADIN],3.0000,200.8600
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,Android App,Android App,2020-03-04,2020-10-18,2020-10-18,2020-03-04,3.0000,1.0000,59.9900,315.9400,[AKTIFSPOR],4.0000,375.9300


In [ ]:
#4. Examine variable types. Convert date variables to date format.

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [30]:
df['first_order_date'] = pd.to_datetime(df['first_order_date'])
df['last_order_date'] = pd.to_datetime(df['last_order_date'])
df['last_order_date_online'] = pd.to_datetime(df['last_order_date_online'])
df['last_order_date_offline'] = pd.to_datetime(df['last_order_date_offline'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

In [31]:
df.head(7)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order,total_customer_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN],5.0000,939.3700
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0000,2.0000,159.9700,1853.5800,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0000,2013.5500
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]",5.0000,585.3200
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]",2.0000,121.9700
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR],2.0000,209.9800
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,Desktop,Offline,2018-11-18,2021-03-13,2018-11-18,2021-03-13,1.0000,2.0000,150.8700,49.9900,[KADIN],3.0000,200.8600
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,Android App,Android App,2020-03-04,2020-10-18,2020-10-18,2020-03-04,3.0000,1.0000,59.9900,315.9400,[AKTIFSPOR],4.0000,375.9300


In [ ]:
#5. Examine the distribution of the number of customers, total number of products purchased, and total spending across different shopping channels.

In [43]:
df.groupby('order_channel').agg({
    'master_id':lambda x: x.count(),
    'total_order': lambda x: x.sum(),
    'total_customer_value': lambda x: x.sum()})

,master_id,total_order,total_customer_value
order_channel,,,
Android App,9495,52269.0000,7819062.7600
Desktop,2735,10920.0000,1610321.4600
Ios App,2833,15351.0000,2525999.9300
Mobile,4882,21679.0000,3028183.1600


In [ ]:
#6. Rank the top 10 customers with the highest revenue.

In [34]:
df.sort_values("total_customer_value", ascending=False).head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order,total_customer_value
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.0000,2.0000,684.9700,45220.1300,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.0000,45905.1000
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,Android App,Android App,2018-08-06,2021-02-23,2021-02-23,2020-07-06,67.0000,1.0000,130.4900,36687.8000,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",68.0000,36818.2900
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.0000,1.0000,1263.7600,32654.3400,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.0000,33918.1000
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,Ios App,Offline,2021-03-01,2021-04-13,2021-03-18,2021-04-13,10.0000,1.0000,538.9400,30688.4700,"[ERKEK, KADIN, AKTIFSPOR]",11.0000,31227.4100
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,Android App,Offline,2021-04-07,2021-04-27,2021-04-07,2021-04-27,1.0000,3.0000,18119.1400,2587.2000,[AKTIFSPOR],4.0000,20706.3400
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.0000,4.0000,843.6800,17599.8900,"[ERKEK, KADIN, AKTIFSPOR]",70.0000,18443.5700
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.0000,1.0000,82.4800,16836.0900,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.0000,16918.5700
163,fef57ffa-aae6-11e9-a2fc-000d3a38a36f,Mobile,Desktop,2016-11-08,2021-05-12,2021-05-12,2020-07-09,36.0000,1.0000,180.7300,12545.3700,"[ERKEK, AKTIFSPOR]",37.0000,12726.1000
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.0000,1.0000,49.9900,12232.2500,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.0000,12282.2400
18767,fc0ce7a4-9d87-11e9-9897-000d3a38a36f,Desktop,Desktop,2018-11-24,2020-11-11,2020-11-11,2019-12-06,18.0000,2.0000,64.9700,12038.1800,"[ERKEK, KADIN]",20.0000,12103.1500


In [ ]:
#7. Rank the top 10 customers with the highest number of orders.

In [35]:
df.sort_values("total_order", ascending=False).head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order,total_customer_value
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.0000,2.0000,684.9700,45220.1300,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.0000,45905.1000
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.0000,1.0000,49.9900,12232.2500,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.0000,12282.2400
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,Android App,Offline,2019-08-07,2020-11-04,2020-09-07,2020-11-04,2.0000,109.0000,10239.4600,143.9800,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",111.0000,10383.4400
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,Android App,Offline,2018-10-18,2020-06-30,2018-10-18,2020-06-30,1.0000,87.0000,8432.2500,139.9800,[ERKEK],88.0000,8572.2300
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,Ios App,Ios App,2019-02-14,2021-04-05,2021-04-05,2020-02-17,2.0000,81.0000,3997.5500,242.8100,[ERKEK],83.0000,4240.3600
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.0000,1.0000,1263.7600,32654.3400,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.0000,33918.1000
9347,44d032ee-a0d4-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2019-02-11,2021-02-11,2021-02-11,2020-12-24,3.0000,74.0000,4640.7700,543.2800,"[KADIN, AKTIFSPOR]",77.0000,5184.0500
10954,b27e241a-a901-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2015-09-12,2021-04-01,2021-04-01,2019-10-08,72.0000,3.0000,292.9300,5004.9500,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",75.0000,5297.8800
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.0000,1.0000,82.4800,16836.0900,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.0000,16918.5700
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.0000,4.0000,843.6800,17599.8900,"[ERKEK, KADIN, AKTIFSPOR]",70.0000,18443.5700


In [ ]:
#8. Functionize the data preprocessing steps.

In [ ]:
#def data_preperation (df):
#  df['total_order'] = df['order_num_total_ever_online'] + df['order_num_total_ever_offline']
#  df['total_customer_value'] = df['customer_value_total_ever_offline'] + df['customer_value_total_ever_online']
#  df['first_order_date'] = pd.to_datetime(df['first_order_date'])
#  df['last_order_date_online'] = pd.to_datetime(df['last_order_date_online'])
#  df['last_order_date'] = pd.to_datetime(df['last_order_date'])
#  df['last_order_date_offline'] = pd.to_datetime(df['last_order_date_offline'])

#  return df

Second Part

In [ ]:
#1. Define Recency, Frequency, and Monetary metrics.
#2. Calculate Recency, Frequency, and Monetary metrics for each customer.
#3. Assign the calculated metrics to a variable named "rfm".
#4. Rename the metrics as recency, frequency, and monetary.

Recency, Frequency, and Monetary (RFM) metrics are commonly used in data science and marketing to analyze customer behavior and segment customers based on their buying habits. 

Recency refers to the amount of time that has passed since a customer's last purchase. This metric is important because customers who have made a purchase recently are more likely to make another purchase than those who have not.

Frequency refers to the number of purchases a customer has made within a given time period. Customers who make frequent purchases are often more loyal and valuable to a business.

Monetary refers to the total amount of money a customer has spent on purchases within a given time period. Customers who have spent more money are often more valuable to a business and may require different marketing strategies compared to customers who have spent less. 

By analyzing these three metrics, businesses can gain insights into customer behavior and segment customers based on their value and potential for future purchases.

In [46]:
df["last_order_date"].max()

Timestamp('2021-05-30 00:00:00')

In [50]:
today_date = dt.datetime(2021,6,1)
print(today_date,type(today_date))

2021-06-01 00:00:00 <class 'datetime.datetime'>


In [148]:
rfm = pd.DataFrame()
rfm["master_id"] = df["master_id"]
rfm["recency"] = (today_date - df["last_order_date"]).astype('timedelta64[D]')
rfm["frequency"] = df["total_order"]
rfm["monetary"] = df["total_customer_value"]

In [149]:
rfm.head(10)

,master_id,recency,frequency,monetary
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.0000,5.0000,939.3700
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.0000,21.0000,2013.5500
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.0000,5.0000,585.3200
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.0000,2.0000,121.9700
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.0000,2.0000,209.9800
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,80.0000,3.0000,200.8600
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,226.0000,4.0000,375.9300
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,293.0000,2.0000,163.6300
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,86.0000,5.0000,1054.6900
9,1143f032-440d-11ea-8b43-000d3a38a36f,240.0000,2.0000,165.9600


In [151]:
df[df['master_id'] == 'cc294636-19f0-11eb-8d74-000d3a38a36f']

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order,total_customer_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN],5.0000,939.3700


In [153]:
rfm.columns = ['master_id','recency','frequency','monetary']

rfm.head(10)

,master_id,recency,frequency,monetary
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.0000,5.0000,939.3700
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.0000,21.0000,2013.5500
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.0000,5.0000,585.3200
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.0000,2.0000,121.9700
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.0000,2.0000,209.9800
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,80.0000,3.0000,200.8600
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,226.0000,4.0000,375.9300
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,293.0000,2.0000,163.6300
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,86.0000,5.0000,1054.6900
9,1143f032-440d-11ea-8b43-000d3a38a36f,240.0000,2.0000,165.9600


In [154]:
# 1. Convert Recency, Frequency, and Monetary metrics into scores between 1-5 using qcut function.
# 2. Save these scores as recency_score, frequency_score, and monetary_score.
# 3. Combine recency_score and frequency_score into a single variable called RF_SCORE.

In [155]:
rfm["recency_score"] = pd.qcut(rfm['recency'],5,labels=[5,4,3,2,1])

In [156]:
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"),5,labels=[1,2,3,4,5])

In [157]:
rfm["monetary_score"] = pd.qcut(rfm['monetary'],5,labels=[1,2,3,4,5])

In [158]:
rfm.head(5)

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.0000,5.0000,939.3700,3,4,4
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.0000,21.0000,2013.5500,3,5,5
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.0000,5.0000,585.3200,2,4,3
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.0000,2.0000,121.9700,3,1,1
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.0000,2.0000,209.9800,3,1,1


In [159]:
rfm["RF_SCORE"] = (rfm['recency_score'].astype(str) + 
                    rfm['frequency_score'].astype(str))

In [160]:
rfm.head(10)

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.0000,5.0000,939.3700,3,4,4,34
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.0000,21.0000,2013.5500,3,5,5,35
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.0000,5.0000,585.3200,2,4,3,24
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.0000,2.0000,121.9700,3,1,1,31
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.0000,2.0000,209.9800,3,1,1,31
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,80.0000,3.0000,200.8600,4,2,1,42
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,226.0000,4.0000,375.9300,2,3,2,23
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,293.0000,2.0000,163.6300,1,1,1,11
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,86.0000,5.0000,1054.6900,3,4,5,34
9,1143f032-440d-11ea-8b43-000d3a38a36f,240.0000,2.0000,165.9600,1,1,1,11


In [161]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,19945.0000,134.4584,103.2811,2.0000,43.0000,111.0000,202.0000,367.0000
frequency,19945.0000,5.0248,4.7427,2.0000,3.0000,4.0000,6.0000,202.0000
monetary,19945.0000,751.2443,895.4022,44.9800,339.9800,545.2700,897.7800,45905.1000


In [162]:
# 1. Define segments for the created RF scores.
# 2. Convert the scores into segments using the seg_map.

In [163]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [164]:
rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)

In [165]:
rfm.head(10)

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.0000,5.0000,939.3700,3,4,4,34,loyal_customers
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.0000,21.0000,2013.5500,3,5,5,35,loyal_customers
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.0000,5.0000,585.3200,2,4,3,24,at_risk
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.0000,2.0000,121.9700,3,1,1,31,about_to_sleep
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.0000,2.0000,209.9800,3,1,1,31,about_to_sleep
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,80.0000,3.0000,200.8600,4,2,1,42,potential_loyalists
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,226.0000,4.0000,375.9300,2,3,2,23,at_risk
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,293.0000,2.0000,163.6300,1,1,1,11,hibernating
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,86.0000,5.0000,1054.6900,3,4,5,34,loyal_customers
9,1143f032-440d-11ea-8b43-000d3a38a36f,240.0000,2.0000,165.9600,1,1,1,11,hibernating


In [166]:
# 1. Examine the averages of recency, frequency, and monetary for each segment.
# 2. Using RFM analysis, find the customers in the relevant profile for the following 2 cases and save their customer IDs as a csv file:
      #   a. FLO is adding a new women's shoe brand to its portfolio. The prices of this brand's products are above the general customer preferences. 
      # Therefore, they want to communicate with the customers who are interested in this profile and make special efforts to promote the brand and sell its products. 
      # The loyal customers (champions, loyal_customers) and those who shop in the women's category will be the targeted customers for communication. 
      # Save the customer IDs of these customers to a csv file.
      #   b. A discount of nearly 40% is planned for men's and children's products. The company wants to target customers who are interested in these categories, 
      # but haven't made a purchase in a long time, as well as new customers. Save the customer IDs of these customers who are in the appropriate profile to a csv file.

In [167]:
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])

recency       frequency        monetary      
                        mean count      mean count      mean count
segment                                                           
about_to_sleep      113.7851  1629    2.4015  1629  359.0090  1629
at_risk             241.6068  3131    4.4724  3131  646.6102  3131
cant_loose          235.4442  1200   10.6983  1200 1474.4682  1200
champions            17.1066  1932    8.9343  1932 1406.6251  1932
hibernating         247.9495  3604    2.3940  3604  366.2671  3604
loyal_customers      82.5948  3361    8.3746  3361 1216.8186  3361
need_attention      113.8287   823    3.7278   823  562.1430   823
new_customers        17.9176   680    2.0000   680  339.9555   680
potential_loyalists  37.1559  2938    3.3043  2938  533.1845  2938
promising            58.9212   647    2.0000   647  335.6727   647

In [168]:
rfm.head(5)

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.0000,5.0000,939.3700,3,4,4,34,loyal_customers
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.0000,21.0000,2013.5500,3,5,5,35,loyal_customers
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.0000,5.0000,585.3200,2,4,3,24,at_risk
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.0000,2.0000,121.9700,3,1,1,31,about_to_sleep
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.0000,2.0000,209.9800,3,1,1,31,about_to_sleep


In [169]:
rfm[(rfm["segment"] == "champions") | (rfm["segment"] == "loyal_customers")].head()

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.0000,5.0000,939.3700,3,4,4,34,loyal_customers
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.0000,21.0000,2013.5500,3,5,5,35,loyal_customers
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,86.0000,5.0000,1054.6900,3,4,5,34,loyal_customers
11,c2e15af2-9eed-11e9-9897-000d3a38a36f,13.0000,20.0000,2861.6000,5,5,5,55,champions
14,241f0ad0-afb5-11e9-9757-000d3a38a36f,2.0000,24.0000,4413.2900,5,5,5,55,champions


In [170]:
rfm[rfm["segment"].isin(["champions", "loyal_customers"])].head()

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.0000,5.0000,939.3700,3,4,4,34,loyal_customers
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.0000,21.0000,2013.5500,3,5,5,35,loyal_customers
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,86.0000,5.0000,1054.6900,3,4,5,34,loyal_customers
11,c2e15af2-9eed-11e9-9897-000d3a38a36f,13.0000,20.0000,2861.6000,5,5,5,55,champions
14,241f0ad0-afb5-11e9-9757-000d3a38a36f,2.0000,24.0000,4413.2900,5,5,5,55,champions


In [171]:
rfm = pd.merge(rfm, df[["master_id", "interested_in_categories_12"]], on="master_id", how="left")

In [182]:
#df.head(5)

df[df["master_id"] == "cc294636-19f0-11eb-8d74-000d3a38a36f"].head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order,total_customer_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN],5.0000,939.3700


In [173]:
rfm.head(5)

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.0000,5.0000,939.3700,3,4,4,34,loyal_customers,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.0000,21.0000,2013.5500,3,5,5,35,loyal_customers,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.0000,5.0000,585.3200,2,4,3,24,at_risk,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.0000,2.0000,121.9700,3,1,1,31,about_to_sleep,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.0000,2.0000,209.9800,3,1,1,31,about_to_sleep,[AKTIFSPOR]


In [174]:
rfm[((rfm["segment"] == "champions") | (rfm["segment"] == "loyal_customers")) & rfm['interested_in_categories_12'].apply(lambda x: 'KADIN' in x)].head()

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.0000,5.0000,939.3700,3,4,4,34,loyal_customers,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.0000,21.0000,2013.5500,3,5,5,35,loyal_customers,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,86.0000,5.0000,1054.6900,3,4,5,34,loyal_customers,"[ERKEK, COCUK, KADIN]"
11,c2e15af2-9eed-11e9-9897-000d3a38a36f,13.0000,20.0000,2861.6000,5,5,5,55,champions,"[COCUK, KADIN, AKTIFSPOR]"
33,fb840306-1219-11ea-a001-000d3a38a36f,116.0000,8.0000,923.6400,3,5,4,35,loyal_customers,"[ERKEK, KADIN, AKTIFSPOR]"


In [175]:
aimdf = rfm[((rfm["segment"] == "champions") | (rfm["segment"] == "loyal_customers")) & rfm['interested_in_categories_12'].apply(lambda x: 'KADIN' in x)]

In [176]:
aimdf.to_csv("aim.csv")

In [177]:
#cant loose them
#about to sleep
#new customers
#hibernating

In [178]:
rfm[rfm["segment"].isin(["cant_loose","about_to_sleep","new_customers","hibernating"])].head()

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment,interested_in_categories_12
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.0000,2.0000,121.9700,3,1,1,31,about_to_sleep,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.0000,2.0000,209.9800,3,1,1,31,about_to_sleep,[AKTIFSPOR]
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,293.0000,2.0000,163.6300,1,1,1,11,hibernating,[COCUK]
9,1143f032-440d-11ea-8b43-000d3a38a36f,240.0000,2.0000,165.9600,1,1,1,11,hibernating,"[KADIN, AKTIFSPOR]"
10,ae608ece-c9d8-11ea-a31e-000d3a38a36f,311.0000,2.0000,114.9800,1,1,1,11,hibernating,"[AKTIFCOCUK, COCUK, KADIN]"


In [179]:
aimdf2 = rfm[rfm["segment"].isin(["cant_loose", "about_to_sleep","new_customers"]) & 
            (rfm['interested_in_categories_12'].str.contains("ERKEK") | (rfm['interested_in_categories_12'].str.contains("COCUK")))]

In [180]:
aimdf2.head()

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment,interested_in_categories_12
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.0000,2.0000,121.9700,3,1,1,31,about_to_sleep,"[AKTIFCOCUK, COCUK]"
15,13ed97a4-b167-11e9-89fa-000d3a38a36f,288.0000,8.0000,932.3600,1,5,4,15,cant_loose,"[AKTIFCOCUK, COCUK, KADIN]"
23,4fbb0cbe-9085-11ea-af25-000d3a38a36f,123.0000,2.0000,242.4500,3,1,1,31,about_to_sleep,"[COCUK, KADIN]"
34,1acf7870-87af-11ea-ace9-000d3a38a36f,103.0000,2.0000,298.0900,3,1,1,31,about_to_sleep,"[ERKEK, KADIN]"
56,92a334ee-58f2-11eb-9e65-000d3a38a36f,110.0000,2.0000,214.8600,3,1,1,31,about_to_sleep,"[ERKEK, KADIN]"


In [181]:
aimdf2.to_csv("aim2.csv")